In [ ]:
#ref.https://github.com/danwagnerco/similarweb

# Import package

In [39]:
import requests
import pandas as pd
from main import GoogleAPIClient
import pandas as pd
import datetime
from datetime import date
import numpy as np
import pymssql 
import pandas as pd
from dateutil.relativedelta import relativedelta

#  Function

In [40]:
class GoogleSheets(GoogleAPIClient):
    def __init__(self) -> None:
        # 呼叫 GoogleAPIClient.__init__()，並提供 serviceName, version, scope
        super().__init__(
            'sheets',
            'v4',
            ['https://www.googleapis.com/auth/spreadsheets'],
        )

    def getWorksheet2Df(self, spreadsheetId: str, range: str):#獲取指定google sheet的檔案，輸出成dataframe
        request = self.googleAPIService.spreadsheets().values().get(
            spreadsheetId=spreadsheetId,
            range=range,
        )
        result = request.execute()['values']
        header = result[0]
        del result[0]
        return pd.DataFrame(result, columns=header)
    
    def getWorksheet(self, spreadsheetId: str, range: str):#獲取指定google sheet的檔案，輸出成list
        request = self.googleAPIService.spreadsheets().values().get(
            spreadsheetId=spreadsheetId,
            range=range,
        )
        response = request.execute()['values']
        return response

    def clearWorksheet(self, spreadsheetId: str, range: str):#清除指定google sheet的資料
        self.googleAPIService.spreadsheets().values().clear(
            spreadsheetId=spreadsheetId,
            range=range,
        ).execute()
        return 'done'
    
    def setWorksheet(self, spreadsheetId: str, range: str, df: pd.DataFrame):#清除指定google sheet的資料後新增資料(dataframe)
        self.clearWorksheet(spreadsheetId, range)
        #使用 update 的語法把我們新的數據加入至 Google Sheets
        self.googleAPIService.spreadsheets().values().update(
            spreadsheetId=spreadsheetId,
            range=range,
            valueInputOption='USER_ENTERED',
            body={
                'majorDimension': 'ROWS',
                'values': df.T.reset_index().T.values.tolist()
            },
        ).execute()
        return 'done'

    def appendWorksheet(self, spreadsheetId: str, range: str, df: pd.DataFrame):#對指定google sheet加入資料(dataframe)
        self.googleAPIService.spreadsheets().values().append(
            spreadsheetId=spreadsheetId,
            range=range,
            valueInputOption='USER_ENTERED',
            body={
                'majorDimension': 'ROWS',
                'values': df.values.tolist()
            },
        ).execute()
        return 'done'
    

In [224]:
def howManyDays(year):#輸入年分，回傳當年度天數
    if year % 4 == 0 and year % 100 != 0:
        return 364
    elif year % 4 == 0 and year % 100 == 0 and year % 400==0:
        return 364
    else:
        return 365
    
def getCompare7DaysToday():#獲得昨天、去年同期和前年同期的日期字串
    yesterday = ((datetime.datetime.now())+datetime.timedelta(days=-1)).strftime('%Y-%m-%d')
    
    day2 = ((datetime.datetime.now())+datetime.timedelta(days=-2)).strftime('%Y-%m-%d')
    
    day3 = ((datetime.datetime.now())+datetime.timedelta(days=-3)).strftime('%Y-%m-%d')
    
    day4 = ((datetime.datetime.now())+datetime.timedelta(days=-4)).strftime('%Y-%m-%d')
    
    day5 = ((datetime.datetime.now())+datetime.timedelta(days=-5)).strftime('%Y-%m-%d')
    
    day6 = ((datetime.datetime.now())+datetime.timedelta(days=-6)).strftime('%Y-%m-%d')
    
    day7 = ((datetime.datetime.now())+datetime.timedelta(days=-7)).strftime('%Y-%m-%d')
    
    
    return yesterday, day2, day3, day4, day5, day6, day7

def setCompare7Days(yyyy, mm, dd):#輸入指定日期，獲得當天、去年同期和前年同期的日期字串
    day = datetime.date(yyyy, mm, dd)
    day1 = ((day)+datetime.timedelta(days=-1)).strftime('%Y-%m-%d')
    day2 = ((day)+datetime.timedelta(days=-2)).strftime('%Y-%m-%d')
    day3 = ((day)+datetime.timedelta(days=-3)).strftime('%Y-%m-%d')
    day4 = ((day)+datetime.timedelta(days=-4)).strftime('%Y-%m-%d')
    day5 = ((day)+datetime.timedelta(days=-5)).strftime('%Y-%m-%d')
    day6 = ((day)+datetime.timedelta(days=-6)).strftime('%Y-%m-%d')
    day7 = ((day)+datetime.timedelta(days=-7)).strftime('%Y-%m-%d')
    return day1, day2, day3, day4, day5, day6, day7

In [209]:
def getDataDf(web, layer, dimen, startM, endM):
    url = "https://api.similarweb.com/v1/website/%s.com.tw/%s/%s?api_key=7e552f683942499fbd752ad182af1dab&start_date=%s&end_date=%s&country=tw&granularity=daily&main_domain_only=false&format=json&show_verified=false&mtd=true"%(web, layer, dimen, startM, endM)
    headers = {"accept": "text/plain"}
    response = requests.get(url, headers=headers)
    dimenFilter = dimen.replace('-', '_')
    falDf = pd.DataFrame(eval(response.text[response.text.find(dimenFilter)+len(dimenFilter)+3:-2]))
    return falDf

def getRankDf(web):
    url = "https://api.similarweb.com/v1/website/%s.com.tw/country-rank/country-rank?api_key=7e552f683942499fbd752ad182af1dab&country=tw&granularity=daily&main_domain_only=false&format=json&show_verified=false&mtd=true"%(web)
    headers = {"accept": "text/plain"}
    response = requests.get(url, headers=headers)
    dimenFilter = 'country-rank'.replace('-', '_')
    falDf = pd.DataFrame(eval(response.text[response.text.find(dimenFilter)+len(dimenFilter)+3:-2]), index=[0])
    return falDf

# 網頁統計數據撈取

In [264]:
url = "https://api.similarweb.com/v1/website/%s.com.tw/%s/%s?api_key=7e552f683942499fbd752ad182af1dab&start_date=%s&end_date=%s&country=tw&granularity=daily&main_domain_only=false&format=json&show_verified=false&mtd=true"%('1111', 'total-traffic-and-engagement', 'visits', startMonth, endMonth)
headers = {"accept": "text/plain"}
response = requests.get(url, headers=headers)
response.content

b'{"meta":{"request":{"granularity":"Daily","main_domain_only":false,"mtd":true,"show_verified":false,"state":null,"format":"json","domain":"1111.com.tw","start_date":"2022-12-01","end_date":"2022-12-25","country":"tw"},"status":"Success","last_updated":"2022-11-30"},"visits":[{"date":"2022-12-01","visits":236471.03670316984},{"date":"2022-12-02","visits":174340.49714494342},{"date":"2022-12-03","visits":140081.11783446523},{"date":"2022-12-04","visits":180838.700126604},{"date":"2022-12-05","visits":210621.43348357102},{"date":"2022-12-06","visits":167072.62267575145},{"date":"2022-12-07","visits":194148.37594915795},{"date":"2022-12-08","visits":203287.30171804337},{"date":"2022-12-09","visits":170705.3918154629},{"date":"2022-12-10","visits":129314.02298120376},{"date":"2022-12-11","visits":180985.78034182233},{"date":"2022-12-12","visits":174868.68751437735},{"date":"2022-12-13","visits":159387.68616303572},{"date":"2022-12-14","visits":191044.17996957904},{"date":"2022-12-15","vis

In [233]:
dateList = [i for i in setCompare7Days(2022, 12, 26)]

#time filter
endMonth = datetime.datetime.strftime(datetime.datetime.strptime(dateList[0], '%Y-%m-%d'),  '%Y-%m')
startMonth = datetime.datetime.strftime(datetime.datetime.strptime(dateList[-1], '%Y-%m-%d'),  '%Y-%m')

## 1111

In [255]:
# #web data
dataDict = dict()

dataDict['visitsDf'] = getDataDf('1111', 'total-traffic-and-engagement', 'visits', startMonth, endMonth)
dataDict['avdDf'] = getDataDf('1111', 'total-traffic-and-engagement', 'average-visit-duration', startMonth, endMonth)
dataDict['ppvDf'] = getDataDf('1111', 'total-traffic-and-engagement', 'pages-per-visit', startMonth, endMonth)
dataDict['brDf'] = getDataDf('1111', 'total-traffic-and-engagement', 'bounce-rate', startMonth, endMonth)

In [262]:
# add time filter
dataWMaskDict = dict()

for key in dataDict.keys():
    dataWMaskDict[key] = dataDict[key][dataDict[key]['date'].isin(dateList)]
    pd.to_datetime(dataWMaskDict[key]['date']).dt.year
    dataWMaskDict[key].insert(0, 'year', year)
    dataWMaskDict[key].insert(2, 'brand', ['1111']*7)

In [263]:
dataWMaskDict[key]

,year,date,brand,bounce_rate
18,2022,2022-12-19,1111,0.420503
19,2022,2022-12-20,1111,0.465251
20,2022,2022-12-21,1111,0.420518
21,2022,2022-12-22,1111,0.466917
22,2022,2022-12-23,1111,0.456686
23,2022,2022-12-24,1111,0.403721
24,2022,2022-12-25,1111,0.449239


In [248]:
dataWMaskDict['avdDf']['year'] = pd.to_datetime(dataWMaskDict['avdDf']['date']).dt.year

18    2022
19    2022
20    2022
21    2022
22    2022
23    2022
24    2022
Name: date, dtype: int64

In [247]:
dataWMaskDict['avdDf']

,date,average_visit_duration,year
18,2022-12-19,276.528282,2022
19,2022-12-20,256.681211,2022
20,2022-12-21,297.217529,2022
21,2022-12-22,262.143665,2022
22,2022-12-23,225.534271,2022
23,2022-12-24,262.609157,2022
24,2022-12-25,225.226079,2022


# 網頁排名撈取

In [238]:
rankDf1111 = getRankDf('1111')
rankDf104 = getRankDf('104')
rankDf518 = getRankDf('518')

In [239]:
pd.concat([rankDf1111, rankDf104, rankDf518])

,date,country_rank
0,2022-12,120.0
0,2022-12,41.0
0,2022-12,403.0
